In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Custom functions
from useful_lca_functions import get_inventory_dataset, init_simple_lca, multi_lcia
from visualisation_functions import heatmap_lca, heatmap_db_comparison

# LCA

## Import LCI and LCIA methods

In [ ]:
bd.projects.set_current("regioinvent_2025")
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff'
RI_DB = 'Regioinvent'

In [ ]:
# From Ecoinvent
# (mineral name: activity name, reference product, location)
INVENTORIES_EI = {
    "Aluminium": ("market for aluminium, primary, ingot", "aluminium, primary, ingot", "IAI Area, North America"),
    "Antimony": ("market for antimony", "antimony", "GLO"),
    "Cadmium": ("market for cadmium", "cadmium", "GLO"),
    "Cobalt": ("market for cobalt oxide", "cobalt oxide", "GLO"),
    "Copper": ("market for copper, cathode", "copper, cathode", "GLO"),
    "Fluorspar": ("market for fluorspar, 97% purity", "fluorspar, 97% purity", "GLO"),
    "Gold": ("market for gold", "gold", "GLO"),
    "Graphite": ("market for graphite", "graphite", "GLO"),
    "Indium": ("market for indium", "indium", "GLO"),
    "Iron (Fe)": ("market for iron ore, crude ore, 46% Fe", "iron ore, crude ore, 46% Fe", "GLO"),
    "Lead": ("market for lead", "lead", "GLO"),
    "Molybdenum": ("market for molybdenum", "molybdenum", "GLO"),
    "Nickel": ("market for nickel, class 1", "nickel, class 1", "GLO"),
    "Niobium (Nb2O5)": ("market for ferroniobium, 66% Nb", "ferroniobium, 66% Nb", "GLO"),
    "Palladium": ("market for palladium", "palladium", "GLO"),
    "Platinum": ("market for platinum", "platinum", "GLO"),
    "Potash (K2O)": ("market for potash salt", "potash salt", "RoW"),
    "Rhodium": ("market for rhodium", "rhodium", "GLO"),
    "Selenium": ("market for selenium", "selenium", "GLO"),
    "Silver": ("market for silver", "silver", "GLO"),
    "Tellurium": ("market for tellurium, semiconductor-grade", "tellurium, semiconductor-grade", "GLO"),
    "Titanium (TiO2)": ("market for titanium dioxide", "titanium dioxide", "RoW"),
    "Uranium (U3O8)": ("market for uranium hexafluoride", "uranium hexafluoride", "RoW"),
    "Zinc": ("market for zinc", "zinc", "GLO") 
}

In [ ]:
# From Regioinvent
# (mineral name: activity name, reference product, location)
INVENTORIES_RI = {
    "Aluminium": ("consumption market for aluminium, primary, ingot", "aluminium, primary, ingot", "CA"),
    "Antimony": ("consumption market for antimony", "antimony", "CA"),
    "Cadmium": ("consumption market for cadmium", "cadmium", "CA"),
    "Cobalt": ("consumption market for cobalt oxide", "cobalt oxide", "CA"),
    "Copper": ("consumption market for copper, cathode", "copper, cathode", "CA"),
    "Fluorspar": ("consumption market for fluorspar, 97% purity", "fluorspar, 97% purity", "CA"),
    "Gold": ("consumption market for gold", "gold", "CA"),
    "Graphite": ("consumption market for graphite", "graphite", "CA"),
    "Indium": ("consumption market for indium", "indium", "CA"),
    "Iron (Fe)": ("consumption market for iron ore, crude ore, 46% Fe", "iron ore, crude ore, 46% Fe", "CA"),
    "Lead": ("consumption market for lead", "lead", "CA"),
    "Molybdenum": ("consumption market for molybdenum", "molybdenum", "CA"),
    "Nickel": ("consumption market for nickel, class 1", "nickel, class 1", "CA"),
    "Niobium (Nb2O5)": ("consumption market for ferroniobium, 66% Nb", "ferroniobium, 66% Nb", "CA"),
    "Palladium": ("consumption market for palladium", "palladium", "CA"),
    "Platinum": ("consumption market for platinum", "platinum", "CA"),
    "Potash (K2O)": ("consumption market for potash salt", "potash salt", "CA"),
    "Rhodium": ("consumption market for rhodium", "rhodium", "CA"),
    "Selenium": ("consumption market for selenium", "selenium", "CA"),
    "Silver": ("consumption market for silver", "silver", "CA"),
    "Tellurium": ("consumption market for tellurium, semiconductor-grade", "tellurium, semiconductor-grade", "CA"),
    "Titanium (TiO2)": ("consumption market for titanium dioxide", "titanium dioxide", "CA"),
    "Uranium (U3O8)": ("consumption market for uranium hexafluoride", "uranium hexafluoride", "CA"),
    "Zinc": ("consumption market for zinc", "zinc", "CA") 
}

In [ ]:
INVENTORIES_EI_ds = get_inventory_dataset(INVENTORIES_EI, database_names=[EI_DB])

In [ ]:
INVENTORIES_RI_ds = get_inventory_dataset(INVENTORIES_RI, database_names=[RI_DB])

In [ ]:
# We can also import some from IW+2.1
expert_ei310 = 'data/IW+2.1/impact_world_plus_21_brightway2_expert_version_ei310.5535d12bedce3770ffef004e84229fd1.bw2package'
bw.BW2Package.import_file(expert_ei310)

In [ ]:
IMPACT_METHODS = {
#'GWP100': ('IPCC 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT'),
'Total HH': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Human health', 'Total human health'), 
'Total EQ': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Ecosystem quality', 'Total ecosystem quality'), 
}

## Perform specific LCA 

## With EI_DB

In [ ]:
# Initialize LCA object
lca_ei = init_simple_lca(INVENTORIES_EI_ds["Aluminium"])

In [ ]:
specific_lca_ei = {}
for rm in INVENTORIES_EI_ds:
    impacts = multi_lcia(lca_ei, INVENTORIES_EI_ds[rm], IMPACT_METHODS)
    specific_lca_ei[rm] = impacts

specific_lca_ei = pd.DataFrame(specific_lca_ei).T
specific_lca_ei = specific_lca_ei.reset_index().rename(columns={
    'index': 'Commodity',
})

In [ ]:
specific_lca_ei

## With Regioinvent

In [ ]:
# Initialize LCA object
lca_ri = init_simple_lca(INVENTORIES_RI_ds["Aluminium"])

In [ ]:
specific_lca_ri = {}
for rm in INVENTORIES_RI_ds:
    impacts = multi_lcia(lca_ri, INVENTORIES_RI_ds[rm], IMPACT_METHODS)
    specific_lca_ri[rm] = impacts

specific_lca_ri = pd.DataFrame(specific_lca_ri).T
specific_lca_ri = specific_lca_ri.reset_index().rename(columns={
    'index': 'Commodity',
})

In [ ]:
specific_lca_ri

## Compare and plot the differences

In [ ]:
heatmap_lca(specific_lca_ei, 
            title='Env impacts with EI markets', 
            save_path='results/production_impacts_2023/env_impacts_market_ei.png')


In [ ]:
heatmap_lca(specific_lca_ri, 
            title='Env impacts with RI markets', 
            save_path='results/production_impacts_2023/env_impacts_markets_ri.png')

In [ ]:
heatmap_db_comparison(specific_lca_ei, 
                      specific_lca_ri, 
                    title="Differences between EI3.10 and Regioinvent", 
                    save_path='results/production_impacts_2023/differences_ei_ri.png')


# Scale Regioinvent with 2023 production 

In [ ]:
canada_production_df_kg = pd.read_csv(r'data/World_mining_data_2024/canada_production_wmd2024_kg.csv')

In [ ]:
# We make sure we have the same number of commodities
canada_production_df_kg["Commodity"].nunique() == specific_lca_ri['Commodity'].nunique()

In [ ]:
df_2023 = pd.merge(canada_production_df_kg, specific_lca_ri, left_on='Commodity', right_on='Commodity')

In [ ]:
df_2023

In [ ]:
# Multiply the production volume by each impact category column
impact_columns = ['Total HH (DALY)', 'Total EQ (PDF.m2.yr)']  # Replace these with actual column names in your impact DataFrame
for col in impact_columns:
    df_2023[f'{col} Impact'] = df_2023['Production Volume (kg)'] * df_2023[col]
df_2023.columns

In [ ]:
df_2023

In [ ]:
# Resulting DataFrame contains original production volume and multiplied impacts
df_2023_result = df_2023[[ 'Commodity', 'Production Volume (kg)', 'World Share (%)', 
                            'Total HH (DALY) Impact',
                            'Total EQ (PDF.m2.yr) Impact']]

In [ ]:
df_2023_result

In [ ]:
df_2023_result.to_csv('results/production_impacts_2023/env_impacts_2023.csv', index=False)

In [ ]:
heatmap_lca(df_2023_result,
            title = '2023 Canadian mineral production and environmental impacts',
            save_path='results/production_impacts_2023/canada_2023_production_impacts_regioinvent.png')